In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("thedevastator/nlp-mental-health-conversations")

print("Path to dataset files:", path)

/home/rochisnu/miniconda3/envs/genai/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 1.48M/1.48M [00:01<00:00, 995kB/s] 

Extracting files...
Path to dataset files: /home/rochisnu/.cache/kagglehub/datasets/thedevastator/nlp-mental-health-conversations/versions/2


In [ ]:
import pandas as pd
data=pd.read_csv(path+"/train.csv")
data

,Context,Response
0,I'm going through some things with my feelings...,"If everyone thinks you're worthless, then mayb..."
1,I'm going through some things with my feelings...,"Hello, and thank you for your question and see..."
2,I'm going through some things with my feelings...,First thing I'd suggest is getting the sleep y...
3,I'm going through some things with my feelings...,Therapy is essential for those that are feelin...
4,I'm going through some things with my feelings...,I first want to let you know that you are not ...
...,...,...
3507,My grandson's step-mother sends him to school ...,Absolutely not! It is never in a child's best ...
3508,My boyfriend is in recovery from drug addictio...,I'm sorry you have tension between you and you...
3509,The birth mother attempted suicide several tim...,"The true answer is, ""no one can really say wit..."
3510,I think adult life is making him depressed and...,How do you help yourself to believe you requir...


## Step 1: Basic EDA

In [132]:
data.isnull().sum()  ## Check for missing values

Context     0
Response    4
dtype: int64

In [89]:
data[data['Response'].isnull()]['Response']

2434    NaN
3007    NaN
3224    NaN
3225    NaN
Name: Response, dtype: object

In [133]:
## Drop the rows with missing information

data.dropna(inplace=True)
data.reset_index(drop=True,inplace=True)

In [134]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3508 entries, 0 to 3507
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Context   3508 non-null   object
 1   Response  3508 non-null   object
dtypes: object(2)
memory usage: 54.9+ KB


## Step 2: Clean the text (Data Preprocessing)

In [ ]:
data['Response'].count()  ## After removal of missing rows

np.int64(3508)

In [135]:
duplicates = data[data['Response'].duplicated(keep=False)]['Response']  ## Finding the duplicates
duplicates

23      Let me start by saying there are never too man...
24      It is never too late to get help and begin mak...
25      You have been through so much and it sounds li...
26      Absolutely not.  I strongly recommending worki...
27      Absolutely not!  In fact, most people have man...
                              ...                        
3502    The thing that confuses a child the most is fo...
3503    Absolutely not! It is never in a child's best ...
3504    I'm sorry you have tension between you and you...
3505    The true answer is, "no one can really say wit...
3506    How do you help yourself to believe you requir...
Name: Response, Length: 2023, dtype: object

In [136]:
duplicates.duplicated().value_counts()  

Response
True     1029
False     994
Name: count, dtype: int64

Note: This first selects all occurrences of duplicate values.

Then:

False = first occurrence of each duplicated value

True = every later occurrence of a duplicate value

In [137]:
## Other way to check Unique values

dup_values = data['Response'].value_counts()[lambda x: x > 1].index.tolist()
len(dup_values)  ## Unique value count

994

In [138]:
data['Response'].count()

np.int64(3508)

In [139]:
data['Response'] = data['Response'].drop_duplicates().reset_index(drop=True)

In [140]:
data['Response'].count()

np.int64(2479)

In [ ]:
# Remove the spaces

data['Response'] = data['Response'].str.strip()
data['Response'] = data['Response'].str.replace(r'\s+', ' ', regex=True)

# Remove basic HTML entities or common artifacts (e.g., &amp;, &gt;)
data['Response'] = data['Response'].str.replace(r'&[a-z]+;', '', regex=True)

# Replace curly quotes with straight quotes
data['Response'] = data['Response'].str.replace(r'[\u201c\u201d]', '"', regex=True)
data['Response'] = data['Response'].str.replace(r'[\u2018\u2019]', "'", regex=True)

# Replace multiple periods (ellipsis) with three periods to standardize
data['Response'] = data['Response'].str.replace(r'\.{2,}', '...', regex=True)

In [ ]:
# Normalize the text

def normalize_text(text):
    # replace punctuation with " <punct> " so spacing is preserved
    text = re.sub(r"([.,!?;:])", r" \1 ", text)
    text = re.sub(r"\s+", " ", text).strip()  # normalize spaces
    # remove punctuation
    text = re.sub(r"[^\w\s]", "", text)
    return text.lower()


In [145]:
final_data=data['Response']
final_data.dropna(inplace=True)
final_data.reset_index(drop=True,inplace=True)
final_data

0       If everyone thinks you're worthless, then mayb...
1       Hello, and thank you for your question and see...
2       First thing I'd suggest is getting the sleep y...
3       Therapy is essential for those that are feelin...
4       I first want to let you know that you are not ...
                              ...                        
2474    Being in this position is tough. If seeking an...
2475    It sounds like you are on the right track. Rec...
2476    Hello.It sounds like you are really concerned ...
2477    Hi. This is an excellent question ! I think th...
2478                             hmm this is a tough one!
Name: Response, Length: 2479, dtype: object

## Step 3: Creating Synthetic Dataset

In [156]:
df = pd.DataFrame({
    "input_text": final_data.apply(normalize_text),   # text without punctuation
    "output_text": final_data                             # original text with punctuation
})

In [157]:
df = df[df["input_text"] != df["output_text"]]
df.reset_index(drop=True,inplace=True)
df

,input_text,output_text
0,if everyone thinks youre worthless then maybe...,"If everyone thinks you're worthless, then mayb..."
1,hello and thank you for your question and see...,"Hello, and thank you for your question and see..."
2,first thing id suggest is getting the sleep yo...,First thing I'd suggest is getting the sleep y...
3,therapy is essential for those that are feelin...,Therapy is essential for those that are feelin...
4,i first want to let you know that you are not ...,I first want to let you know that you are not ...
...,...,...
2473,being in this position is tough if seeking an...,Being in this position is tough. If seeking an...
2474,it sounds like you are on the right track rec...,It sounds like you are on the right track. Rec...
2475,hello it sounds like you are really concerned...,Hello.It sounds like you are really concerned ...
2476,hi this is an excellent question i think tha...,Hi. This is an excellent question ! I think th...


In [ ]:
# import re
# import pandas as pd
# from typing import List, Tuple, Dict

# # --- 1. Define Punctuation Labels ---
# # These are the labels your model will learn to predict.
# PUNCT_LABELS: Dict[str, str] = {
#     '.': 'PERIOD',
#     ',': 'COMMA',
#     '?': 'QUESTION_MARK',
#     '!': 'EXCLAMATION_MARK',
#     ':': 'COLON',
#     ';': 'SEMICOLON',
#     '--': 'DASH',
#     '---': 'DASH', # Normalizing common punctuation variations
#     '...': 'ELLIPSIS_PERIOD', # Explicitly labeling ellipsis
# }
# DEFAULT_LABEL = 'NONE'

# # Create a set of punctuation tokens for fast lookup
# PUNCTUATION_TOKENS = set(PUNCT_LABELS.keys())

# # Compile a regex pattern to reliably separate words from punctuation.
# # It captures: 
# # 1. Words (including standard contractions like 'isn't', 'i'm')
# # 2. All defined punctuation sequences (e.g., '.', ',', '...')
# PUNCTUATION_PATTERN = r"(\w[\w']*\w|\w|[.,?!:;]{1,}|---?|\.{3})"
# # Note: The pattern is applied with re.findall, which returns all matches.


# def generate_synthetic_dataset(cleaned_responses: List[str]) -> List[Tuple[str, str]]:
#     """
#     Transforms fully punctuated, cleaned text into a sequence labeling dataset.
#     Each word is paired with the punctuation label that should follow it.
    
#     This function processes the text in a generic manner using regex tokenization 
#     to separate words and punctuation tokens, then aligns them.

#     Args:
#         cleaned_responses: A list of strings (your cleaned 'Response' column data).
#                            The text must be lowercased and have normalized whitespace.

#     Returns:
#         A list of (word, label) tuples, ready for model training.
#     """
#     labeled_data: List[Tuple[str, str]] = []

#     for response in cleaned_responses:
#         # Step 1: Tokenize the text into words and punctuation
#         # re.findall returns a list of all matching groups.
#         tokens = re.findall(PUNCTUATION_PATTERN, response.lower())
        
#         # Step 2: Iterate through tokens to assign labels
#         i = 0
#         while i < len(tokens):
#             current_token = tokens[i]
            
#             # Simple check to see if the token is a word (not in our defined punctuation set)
#             # This is a generic approach assuming any token not recognized as punctuation is a word.
#             if current_token not in PUNCTUATION_TOKENS:
                
#                 # Default label is NONE (no punctuation after this word)
#                 label = DEFAULT_LABEL 
                
#                 # Look ahead for the next token (potential punctuation)
#                 next_i = i + 1
                
#                 if next_i < len(tokens):
#                     next_token = tokens[next_i]
                    
#                     # Check if the next token is a defined punctuation mark
#                     if next_token in PUNCTUATION_TOKENS:
#                         label = PUNCT_LABELS[next_token]
#                         # Consume the punctuation token by advancing the index past it
#                         i += 1 
                
#                 # Add the word and its assigned label to the dataset
#                 labeled_data.append((current_token, label))
            
#             # Advance the index to the next token
#             i += 1
            
#     return labeled_data